### 3 - Why you should use Pydantic - Aula

Fernanda de Castro Fernandes


A aula apresenta 4 níveis de validação de usuário, começando do mais básico e simples, até a criação de uma API utilizando o framework FastAPI para realizar tal tarefa.

In [19]:
%pip install pydantic
%pip install pydantic[email]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


##### Exemplo 1

No primeiro exemplo, o usuário é modelado e seu esquema utiliza Pydantic para a definição do esquema e tipagem das variáveis. Ainda é simples, então o usuário não possui papel por padrão inicial.

In [1]:
from enum import auto, IntFlag
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    SecretStr,
    ValidationError
)

In [2]:
class Role(IntFlag):
    Author = auto()
    Editor = auto()
    Developer = auto()
    Admin = Author | Editor | Developer

class User(BaseModel):
    name: str = Field(examples=['Fernanda'])
    email: EmailStr = Field(
        examples=['exemplo@dominio.com'],
        description = 'Email do usuário',
        frozen = True,
    )
    password: SecretStr = Field(
        examples=['Password@123'],
        description = 'Senha do usuário',
    )
    role: Role = Field(
        default=None, 
        description='Função do usuário',
    ) 

A validação compara os dados dos usuários recebidos na forma de um dicionário e as valida com instâncias prontas do Pydantic, sem trabalhar casos ou erros. Aqui qualquer input semelhante com o exemplo definido na classe é aceito.

In [3]:
def validate(data: dict[str, Any]) -> User:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print('User invalido')
        for error in e.errors():
            print(error)

def main() -> None:
    good_data = {
        'name': 'Fernanda',
        'email': 'exemplo@dominio.com',
        'password': 'Password@123',
    }
    bad_data = {'email': '<bad data>','password': '<bad data>'},
    
    validate(good_data)
    validate(bad_data)

if __name__ == '__main__':
    main()

name='Fernanda' email='exemplo@dominio.com' password=SecretStr('**********') role=None
User invalido
{'type': 'model_type', 'loc': (), 'msg': 'Input should be a valid dictionary or instance of User', 'input': ({'email': '<bad data>', 'password': '<bad data>'},), 'ctx': {'class_name': 'User'}, 'url': 'https://errors.pydantic.dev/2.10/v/model_type'}


##### Exemplo 2

Neste exemplo, subimos a barra um pouco, adicionando uma Expressão Regular para validação do nome e da senha, assim como vemos nos sites atuais. "A senha deve conter números, caracteres especiais, letras maiusculas"

In [4]:
import enum
import hashlib
import re
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator,
    SecretStr,
    ValidationError,
)

In [5]:
VALID_PASSWORD_REGEX = re.compile(r'^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$')
VALID_NAME_REGEX = re.compile(r'^[a-zA-Z]{2,}$')

In [6]:
class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8


class User(BaseModel):
    name: str = Field(examples=['Fernanda'])
    email: EmailStr = Field(
        examples = ['user@dominio.com'],
        description = 'O email do usuário',
        frozen = True,
    )
    password: SecretStr = Field(
        examples = ['Password123'], 
        description = 'A senha do usuário',
    )
    role: Role = Field(
        default = None, 
        description = 'O papel do usuário', 
        examples = [1, 2, 4, 8]
    )

Também utilizamos a estrutura ENUM para especificar os papéis possíveis de usuário. Caso seja especificado qualquer outro role fora deles, haverá um erro. Há multiplas funções de validação com decoradores especificando quais campos a função de validação está implementando.

In [7]:
@field_validator('name')
@classmethod
def validate_name(cls, v: str) -> str:
    if not VALID_NAME_REGEX.match(v):
        raise ValueError(
            'Nome inválido, deve conter apenas letras e ter pelo menos 2 caracteres'
        )
    return v

@field_validator('role', mode='before')
@classmethod
def validate_role(cls, v: int | str | Role) -> Role:
    op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
    try:
            return op[type(v)](v)
    except (KeyError,ValueError):        
        raise ValueError(
            f'Função inválida, favor usar um dos valores: {"," .join(x.name for x in Role)}'
        )

@field_validator('role', mode='before')
@classmethod
def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
    if 'name' not in v or 'password' not in v:
        raise ValueError('Nome e senha são obrigatórios')
    if v['name'].casefold() in v['password'].casefold():
        raise ValueError('Senha não pode conter o nome do usuário')
    if not VALID_PASSWORD_REGEX.match(v['password']):
        raise ValueError(
            'Senha inválida, deve conter pelo menos 8 caracteres, uma letra maiúscula, uma letra minúscula e um número'
        )
    v['password'] = hashlib.sha256(v['password'].encode()).hexdigest()
    return v

def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print('Usuário inválido')
        print(e)

Teste com múltiplos casos de dados errados

In [8]:
def main() -> None:
    test_data = dict(
        good_data={
            'name': 'Fernanda',
            'email': 'exemplo@dominio.com',
            'password': 'Password@123',
            'role': 'Admin',
        },
        bad_role={
            'name': 'Fernanda',
            'email': 'exemplo@dominio.com',
            'password': 'Password123',
            'role': 'Programmer',
        },
        bad_data={
            'name': 'Fernanda',
            'email': 'bad email',
            'password': 'bad password',
        },
        bad_name={
            'name': 'Fernanda<-_->',
            'email': 'exemplo@dominio.com',
            'password': 'Password123',
        },
        duplicate={
            'name': 'Fernanda',
            'email': 'exemplo@dominio.com',
            'password': 'Fernanda123',
        },
        missing_data={
            'email': '<bad data>',
            'password': '<bad data>',
        },
    )

    for example_name, data in test_data.items():
        print(example_name)
        validate(data)
        print()


if __name__ == '__main__':
    main()

good_data
Usuário inválido
1 validation error for User
role
  Input should be 1, 2, 4 or 8 [type=enum, input_value='Admin', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/enum

bad_role
Usuário inválido
1 validation error for User
role
  Input should be 1, 2, 4 or 8 [type=enum, input_value='Programmer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/enum

bad_data
Usuário inválido
1 validation error for User
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='bad email', input_type=str]

bad_name
name='Fernanda<-_->' email='exemplo@dominio.com' password=SecretStr('**********') role=None

duplicate
name='Fernanda' email='exemplo@dominio.com' password=SecretStr('**********') role=None

missing_data
Usuário inválido
2 validation errors for User
name
  Field required [type=missing, input_value={'email': '<bad data>', 'password': '<bad data>'}, input_type

##### Exemplo 3

Ainda subindo o nível, garantimos aqui que o papel seja validado, que os dados de criação de um usuário sejam válidos ANTES e DEPOIS da criação do mesmo no sistema.

In [9]:
import enum
import hashlib
import re
from typing import Any, Self
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_serializer,
    field_validator,
    model_serializer,
    model_validator,
    SecretStr,
)

In [10]:
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

In [11]:
class Role(enum.IntFlag):
    User = 0
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

class User(BaseModel):
    name: str = Field(examples=['Exemplo'])
    email: EmailStr = Field(
        examples=['user@dominio.com'],
        description='O email do usuário',
        frozen=True,
    )
    password: SecretStr = Field(
        examples=['Password123'], 
        description='A senha do usuário', 
        exclude=True
    )
    role: Role = Field(
        description='O papel do usuário',
        examples=[1, 2, 4, 8],
        default=0,
        validate_default=True,
    )

In [12]:
@field_validator("name")
def validate_name(cls, v: str) -> str:
    if not VALID_NAME_REGEX.match(v):
        raise ValueError(
            'Nome inválido, deve conter apenas letras e ter pelo menos 2 caracteres'
        )
    return v

@field_validator("role", mode="before")
@classmethod
def validate_role(cls, v: int | str | Role) -> Role:
    op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
    try:
        return op[type(v)](v)
    except (KeyError, ValueError):
        raise ValueError(
            f'Função inválida, favor usar um dos valores: {", ".join(x.name for x in Role)}'
        )
    
@field_validator("role", mode="before")
@classmethod
def validate_user_pre(cls, v:dict[str, Any]) -> dict[str, Any]:
    if 'name' not in v or 'password' not in v:
        raise ValueError('Nome e senha são obrigatórios')
    if v['name'].casefold() in v['password'].casefold():
        raise ValueError('Senha não pode conter o nome do usuário')
    if not VALID_PASSWORD_REGEX.match(v['password']):
        raise ValueError(
            'Senha inválida, deve conter pelo menos 8 caracteres, uma letra maiúscula, uma letra minúscula e um número'
        )
    v['password'] = hashlib.sha256(v['password'].encode()).hexdigest()
    return v

@model_validator(mode='after')
def validate_user_post(self, v: Any) -> Self:
    if self.role == Role.Admin and self.name != 'Fernanda':
        raise ValueError('A única admin é a Fernanda')
    return self

@field_serializer('role', when_used='json')
@classmethod
def serialize_role(cls, v) -> str:
    return v.name

@model_serializer(mode='wrap', when_used='json')
def serialize_user(self, serializer, info) -> dict[str, Any]:
    if not info.include and not info.exclude:
        return {'name': self.name, 'role': self.role.name}
    return serializer(self)

Acima, há a serialização dos papeis para JSON. Ao invés de retornar um 'role.admin', retornará 'admin.

In [13]:
def main() -> None:
    data = {
        'name': 'Fernanda',
        'email': 'exemplo@dominio.com',
        'password': 'Password123',
        'role': '4', # no repositório, o valor é 'Admin'
        }
    user = User.model_validate(data)
    if user:
        print(
            'O serializer que retorna um dicionário:',
            user.model_dump(),
            sep='\n',
            end='\n\n',
        )
        print(
            'O serializer que retorna uma string JSON:',
            user.model_dump(mode='json'),
            sep='\n',
            end='\n\n',
        )
        print(
            'O serializer que retorna uma string JSON, excluindo a papel:',
            user.model_dump(mode='json', exclude=['role']),
            sep='\n',
            end='\n\n', 
        )
        print(
            'O serializer que codifica todos os valores em um dicionario:',
            user.model_dump(mode='json', exclude=['role']),
            sep='\n',
            end='\n\n',
        )

if __name__ == '__main__':
    main()

O serializer que retorna um dicionário:
{'name': 'Fernanda', 'email': 'exemplo@dominio.com', 'role': <Role.Admin: 4>}

O serializer que retorna uma string JSON:
{'name': 'Fernanda', 'email': 'exemplo@dominio.com', 'role': 4}

O serializer que retorna uma string JSON, excluindo a papel:
{'name': 'Fernanda', 'email': 'exemplo@dominio.com'}

O serializer que codifica todos os valores em um dicionario:
{'name': 'Fernanda', 'email': 'exemplo@dominio.com'}



##### Exemplo 4

Por fim, a criação de uma API para validação.

In [14]:
%pip install "fastapi[standard]"

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [15]:
from datetime import datetime
from typing import Optional
from uuid import uuid4

from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

app = FastAPI()

In [16]:
class User(BaseModel):
    model_config = {
        'extra': 'forbid',
    }
    __users__ = []
    name: str = Field(..., description='Nome do usuário')
    email: EmailStr = Field(..., description='Email do usuário')
    friends: list[UUID4] = Field(
        default_factory=list, max_items=500, description='Lista de amigos'
    )
    blocked: list[UUID4] = Field(
        default_factory=list, max_items=500, description='Lista de usuários bloqueados'
    )
    signup_ts: Optional[datetime] = Field(
        default_factory=datetime.now, description= 'Horário de cadastro', kw_only=True
    )
    id: UUID4 = Field(
        default_factory=uuid4, description='Identificador único', kw_only=True
    )

    @field_serializer('id', when_used='json')
    def serialize_id(self, id: UUID4) -> str:
        return str(id)


In [17]:
@app.get('/users', response_model=list[User])
async def get_users() -> list[User]:
    return list(User.__users__)

@app.post('/users', response_model=User)
async def create_user(user: User):
    User.__users__.append(user)
    return user

@app.get('/users/{user_id}', response_model=User)
async def get_user(user_id: UUID4) -> User | JSONResponse:
    try:
        return next((user for user in User.__users__ if user.id == user_id))
    except StopIteration:
        return JSONResponse(status_code=404, content={'message': 'Usuário não encontrado'})

In [18]:
def main() -> None:
    with TestClient(app) as client:
        for i in range(5):
            response = client.post(
                '/users',
                json={
                    'name': f'User {i}',
                    'email': f'exemplo{i}@dominio.com'},
            )
            assert response.status_code == 200
            assert (
                response.json()['name'] == f'User {i}'
            ), 'O nome do usuário deveria ser User {i}'

            user = User.model_validate(response.json())
            assert str(user.id) == response.json()['id'], 'O id do usuário deveria ser o mesmo'
            assert user.signup_ts, 'O horário de cadastro deveria ser preenchido'
            assert user.friends == [], 'A lista de amigos deveria estar vazia'
            assert user.blocked == [], 'A lista de bloqueados deveria estar vazia'

        response = client.get('/users')
        assert response.status_code == 200, 'Codigo da resposta deveria ser 200'
        assert len(response.json()) == 5, 'Deveria ter 5 usuários'

        response = client.post(
            '/users',
            json={
                'name': 'User 5',
                'email': 'exemplo5@dominio.com'},
        )
        assert response.status_code == 200
        assert (
            response.json()['name'] == 'User 5'
        ), 'O nome do usuário deveria ser User 5'
        assert response.json()['id'], 'O usuário deveria ter um id'

        user = User.model_validate(response.json())
        assert str(user.id) == response.json()['id'], 'O id do usuário deveria ser o mesmo'
        assert user.signup_ts, 'O horário de cadastro deveria ser preenchido'
        assert user.friends == [], 'A lista de amigos deveria estar vazia'
        assert user.blocked == [], 'A lista de bloqueados deveria estar vazia'

        response = client.get(f"/users/{response.json()['id']}")
        assert response.status_code == 200
        assert response.json()['name'] == 'User 5',( 
            'O nome do usuário deveria ser User 5')
        
        response = client.get(f'/users/{uuid4()}')
        assert response.status_code == 404
        assert response.json()['message'] == 'Usuário não encontrado', (
            'Não deveria ser possível encontrar esse usuário'
        )

        response = client.post('/users', json={'name': 'User 6', 'email': 'wrong'})
        assert response.status_code == 422, 'O email deveria ser inválido'

if __name__ == '__main__':
    main()